Robb Chun, Vincent Ying

ECEN 523 - Final Project

Jun 3, 2024

In [48]:
!pip install llama-index llama-index-embeddings-huggingface llama_index-llms-ollama

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [49]:
# NOTE: This is ONLY necessary in jupyter notebook.
# Details: Jupyter runs an event-loop behind the scenes.
#          This results in nested event-loops when we start an event-loop to make async queries.
#          This is normally not allowed, we use nest_asyncio to allow it for convenience.
import nest_asyncio

nest_asyncio.apply()

In [2]:
from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    StorageContext,
    load_index_from_storage,
)
from llama_index.core import Settings
from llama_index.core.agent import ReActAgent
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.ollama import Ollama

In [15]:
# Set up OpenAI
import os
import getpass
import openai

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")
openai.api_key = os.environ["OPENAI_API_KEY"]

OpenAI API Key:········


# RAG Search on An Academic Paper in Llama-Index

## Query Engines

https://docs.llamaindex.ai/en/v0.10.34/examples/cookbooks/llama3_cookbook/#create-queryengines

In [16]:
# Set LLM
embed_model = HuggingFaceEmbedding(model_name='sentence-transformers/msmarco-distilbert-base-v4')
llm = Ollama(model="llama3", temperature=0)

In [17]:
# Load documents
background_docs = SimpleDirectoryReader(input_files=["./data/pubmed_background.json"]).load_data()
reference_docs = SimpleDirectoryReader(input_files=["./data/pubmed_reference.json"]).load_data()

# Create Indices
background_index = VectorStoreIndex.from_documents(background_docs, embed_model=embed_model)
reference_index = VectorStoreIndex.from_documents(reference_docs, embed_model=embed_model)

print('Created background and reference vector DBs') 

Created background and reference vector DBs


In [18]:
# Create QueryEngines
background_engine = background_index.as_query_engine(                                                
                                                    response_mode="tree_summarize",
                                                    verbose=True,
                                                    similarity_top_k=10)
reference_engine = reference_index.as_query_engine(                                                
                                                    response_mode="tree_summarize",
                                                    verbose=True,
                                                    similarity_top_k=10)

In [19]:
# Define QueryEngine Tools
query_engine_tools = [
    QueryEngineTool(
        query_engine=background_engine,
        metadata=ToolMetadata(
            name="background_docs",
            description=(
                "Provides background and topic information about medical research."
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
    QueryEngineTool(
        query_engine=reference_engine,
        metadata=ToolMetadata(
            name="reference_docs",
            description=(
                "Provides reference and specific information about medical research."
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
]

In [20]:
# Create ReAct Agent using RAG QueryEngine Tools
agent = ReActAgent.from_tools(
    query_engine_tools,
    llm=llm,
    verbose=True,
)

In [21]:
# Querying
question = "What is a good topic for new medical research?"
response = agent.chat(question)
print(response)

ReadTimeout: timed out

In [10]:
print(response.source_nodes[0].text)

", "article_abstract": "microMend, a novel microstaple skin closure device, may be able to close simple lacerations. This study aimed to evaluate the feasibility and acceptability of using microMend to close these wounds in the ED.", "pub_date": {"year": "2023", "month": "05", "day": "23"}}, {"article_title": "Cold anesthesia for pain reduction during intralesional steroid injection for nodulocystic acne.", "article_abstract": "In any dermatologic procedure, patient acceptance of treatment is heavily influenced by intraprocedural pain. Intralesional triamcinolone injections are very important in keloid scar and nodulocystic acne treatment. However, the main problem of needle-stick procedures is pain. Cryoanesthesia is ideally intended to cool only the epidermis during treatment and has advantage which did not require application time.", "pub_date": {"year": "2023", "month": "05", "day": "23"}}, {"article_title": "Efficacy of certolizumab pegol across baseline rheumatoid factor subgroup

In [11]:
question = "who are the authors of the Retrieval-Augmented Generation (aka RAG) paper? The answer should be on the first page."
response = query_engine.query(question)
print(response)

Thought: I need to find the authors of the Retrieval-Augmented Generation (RAG) paper.
Action: query_engine_tool
Action Input: {'input': 'Who are the authors of the Retrieval-Augmented Generation (RAG) paper?', 'num_beams': 5}
Observation: Error: timed out
Thought: The tool didn't return a response, so I'll try again.
Action: query_engine_tool
Action Input: {'input': 'Who are the authors of the Retrieval-Augmented Generation (RAG) paper?', 'num_beams': 5}
Observation: Error: timed out
Thought: The tool still didn't return a response, so I'll try again.
Action: query_engine_tool
Action Input: {'input': 'Who are the authors of the Retrieval-Augmented Generation (RAG) paper?', 'num_beams': 5}
Observation: Error: timed out
Thought: The tool still didn't return a response, so I'll try again.
Action: query_engine_tool
Action Input: {'input': 'Who are the authors of the Retrieval-Augmented Generation (RAG) paper?', 'num_beams': 5}
Observation: Error: timed out
Observation: Error: Could not pa

KeyboardInterrupt: 

In [ ]:
response